<a href="https://colab.research.google.com/github/Aryan-Shah05/NLP_LAB/blob/main/Assignment_3/NLP_LAB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
words = []
with open('/content/brown_nouns.txt', 'r') as f:
    for line in f:
        word = line.strip()
        if word:
            words.append(word)

print(f"Loaded {len(words)} words.")

Loaded 202793 words.


In [ ]:
class TrieNode:
    """Represents a node in a trie."""
    def __init__(self):
        self.children = {}
        self.frequency = 0

class Trie:
    """Represents a trie (prefix or suffix)."""
    def __init__(self, is_suffix_trie=False):
        self.root = TrieNode()
        self.is_suffix_trie = is_suffix_trie

    def insert(self, word):
        """Inserts a word into the trie."""
        if self.is_suffix_trie:
            word = word[::-1] # Reverse the word for suffix trie

        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.frequency += 1


In [ ]:
prefix_trie = Trie()
suffix_trie = Trie(is_suffix_trie=True)

for word in words:
    prefix_trie.insert(word)
    suffix_trie.insert(word)

print("Words inserted into both prefix and suffix tries.")

Words inserted into both prefix and suffix tries.


In [ ]:
def find_stem_suffix(word, trie):
    """
    Finds the stem and suffix of a word using a given trie.

    Args:
        word: The input word.
        trie: The trie (prefix or suffix) to use for analysis.

    Returns:
        A tuple containing the stem and suffix of the word.
    """
    if trie.is_suffix_trie:
        word = word[::-1]

    node = trie.root
    potential_splits = []
    current_prefix = ""

    for i, char in enumerate(word):
        if char in node.children:
            node = node.children[char]
            current_prefix += char
            # Check for branching: more than one child or end of a word (frequency > 0)
            if len(node.children) > 1 or node.frequency > 0:
                potential_splits.append((i + 1, node.frequency)) # Store index after split and frequency
        else:
            # If a character is not found, stop traversal
            break

    best_split_index = len(word) # Default to no split (whole word is stem)
    best_split_frequency = -1

    # Find the split point with the highest frequency
    for split_index, frequency in potential_splits:
        if frequency > best_split_frequency:
            best_split_frequency = frequency
            best_split_index = split_index

    if trie.is_suffix_trie:
        # For suffix trie, the split index is from the reversed word
        # We need to convert it back to the original word's index
        original_split_index = len(word) - best_split_index
        stem = word[best_split_index:][::-1] # Reverse back to original
        suffix = word[:best_split_index][::-1] # Reverse back to original
    else:
        stem = word[:best_split_index]
        suffix = word[best_split_index:]

    return stem, suffix


In [ ]:
prefix_stemming_results = []
suffix_stemming_results = []

for word in words:
    prefix_stem, prefix_suffix = find_stem_suffix(word, prefix_trie)
    prefix_stemming_results.append((prefix_stem, prefix_suffix))

    suffix_stem, suffix_suffix = find_stem_suffix(word, suffix_trie)
    suffix_stemming_results.append((suffix_stem, suffix_suffix))

print("Stemming applied to all words using both tries.")

Stemming applied to all words using both tries.


In [ ]:
# 1. Iterate and compare results, count differences
differing_results_count = 0
differing_examples = []
sample_size = 20

for i in range(len(words)):
    prefix_result = prefix_stemming_results[i]
    suffix_result = suffix_stemming_results[i]

    if prefix_result != suffix_result:
        differing_results_count += 1
        if len(differing_examples) < sample_size:
            differing_examples.append({
                "word": words[i],
                "prefix_stem_suffix": prefix_result,
                "suffix_stem_suffix": suffix_result
            })

print(f"Total number of words with differing stemming results: {differing_results_count}")

# 3. Analyze a sample of differing cases
print("\nAnalysis of Sample Differing Cases:")
for example in differing_examples:
    print(f"Word: {example['word']}")
    print(f"  Prefix Trie Result: Stem='{example['prefix_stem_suffix'][0]}', Suffix='{example['prefix_stem_suffix'][1]}'")
    print(f"  Suffix Trie Result: Stem='{example['suffix_stem_suffix'][0]}', Suffix='{example['suffix_stem_suffix'][1]}'")
    print("-" * 20)

# 4 & 5. Summarize observations and discuss frequency/probability influence
print("\nSummary of Observations and Discussion:")
print(f"Out of {len(words)} words, {differing_results_count} ({differing_results_count/len(words)*100:.2f}%) had different stemming results between the prefix and suffix tries.")
print("Based on the sample analysis:")
print("- The differences often occur at potential morpheme boundaries where one trie identifies a shorter or longer stem than the other.")
print("- The prefix trie tends to split at the first significant branching point encountered from the beginning of the word (prefix).")
print("- The suffix trie tends to split at the first significant branching point encountered from the end of the word (suffix).")
print("- The frequency measure at branching points heavily influences the split. A high frequency at a node suggests that the path leading to it is a common word or stem, making it a likely split point.")
print("- Differences arise when the most frequent branching point is encountered at different positions depending on whether traversing from the front (prefix) or back (suffix) of the word.")
print("- Subjectively, neither trie consistently produces 'better' results across all cases. The 'correct' stem/suffix split can be ambiguous, and the performance depends on the specific word and its morphological structure relative to the trie's learned frequencies.")
print("- For words with clear suffixes (e.g., plurals like 'cats'), both tries might agree. For words with less clear or multiple potential splits, the results diverge based on the trie's structure and frequencies.")

Total number of words with differing stemming results: 199204

Analysis of Sample Differing Cases:
Word: investigation
  Prefix Trie Result: Stem='investigation', Suffix=''
  Suffix Trie Result: Stem='', Suffix='investigation'
--------------------
Word: primary
  Prefix Trie Result: Stem='primary', Suffix=''
  Suffix Trie Result: Stem='', Suffix='primary'
--------------------
Word: election
  Prefix Trie Result: Stem='election', Suffix=''
  Suffix Trie Result: Stem='', Suffix='election'
--------------------
Word: evidence
  Prefix Trie Result: Stem='evidence', Suffix=''
  Suffix Trie Result: Stem='', Suffix='evidence'
--------------------
Word: irregularities
  Prefix Trie Result: Stem='irregularities', Suffix=''
  Suffix Trie Result: Stem='irregulari', Suffix='ties'
--------------------
Word: place
  Prefix Trie Result: Stem='place', Suffix=''
  Suffix Trie Result: Stem='', Suffix='place'
--------------------
Word: jury
  Prefix Trie Result: Stem='jury', Suffix=''
  Suffix Trie Result

In [ ]:
# Iterate through the original list of words and the prefix trie results
for i in range(len(words)):
    word = words[i]
    stem, suffix = prefix_stemming_results[i]
    # Format and print the output
    print(f"{word}={stem}+{suffix}")

Streaming output truncated to the last 5000 lines.
street=street+
clothes=clothes+
release=release+
head=head+
moments=moment+s
clothes=clothes+
bench=bench+
shoe=shoe+
plenty=plenty+
guys=guy+s
leagues=league+s
guys=guy+s
club=club+
hell=hell+
right=right+
thing=thing+
thing=thing+
job=job+
job=job+
outfielders=outfield+ers
way=way+
interest=interest+
product=product+
insularity=insularity+
reading=reading+
religion=religion+
subject=subject+
religion=religion+
tomes=tomes+
papers=paper+s
birthday=birth+day
store=store+
mother=mother+
image=image+
god=god+
day=day+
mother=mother+
present=present+
gift=gift+
room=room+
dinner=dinner+
night=night+
god=god+
father=father+
laughter=laughter+
mother=mother+
father=father+
parents=parents+
religion=religion+
interest=interest+
study=study+
religion=religion+
interest=interest+
affairs=affairs+
future=future+
opportunity=opportunity+
school=school+
parents=parents+
foundation=foundation+
life=life+
studies=studies+
student=student+
remotenes